# 機械学習をPythonで実践する-21　　～ ハイパーパラメータのチューニング ～

In [101]:
%load_ext autoreload
%autoreload 2
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import itertools
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OrdinalEncoder, LabelEncoder, OneHotEncoder
# # import statsmodels.api as sma
from sklearn.model_selection import train_test_split ,cross_val_score, KFold, RepeatedKFold,StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, log_loss, confusion_matrix,ConfusionMatrixDisplay, \
accuracy_score, precision_score, recall_score,precision_recall_curve,f1_score,roc_curve,auc,get_scorer_names,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from lightGBM_cv import lightGBM_classifier_cv_func
from category_encoders import TargetEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.datasets import fetch_california_housing
from GetDummies import GetDummies
from scipy.stats import uniform
# random.randintとごっちゃになるので下記のように定義することがよくある。
from scipy.stats import randint as sp_randint
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

%matplotlib inline
import matplotlib.pyplot as plt


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Grid Search
練習用としてsklearnのcalifornia housingのデータセットを用いる。  
カリフォルニア州の家の値段の中央値を目的変数とするデータセットである。  
今回はLightGBMの下記ハイパーパラメタに対してチューニングしてみる。  
* num_leaves（決定木の葉の数）: [10, 20, 30, 40, 50, 60]※
* max_depth（決定木の深さ）: [5, 10, 15]
* reg_alpha（L1正則化項のλ）: [0, 0.01, 0.03]  

基本的にLightGBMではearly stoppingを使用するので、n_estimators（決定木の数）のチューニングは不要。  
early stoppingする前に学習が終わらない程度に大きな値を指定しておけばよい。  
learning rate（学習率）は一般的に低い方が精度が良いので、チューニング時は固定して  
全学習データを使ったモデル構築時など最終的なモデルを構築する際に低い値を試せばよい。  
このようにハイパーパラメータの意味・用法を考慮することでチューニング対象のハイパーパラメータを減らし、計算時間も減らすことが出来る。  

※
決定木は二分木なので1分岐あたりの葉の数は2となる。すなわち全体では2の木の深さ乗以下の葉の数になる。  
max_depthが5の場合、葉の数は2^5=32以下になるので、max_depht=5とnum_leaves=[40,50,60]の組み合わせはあり得ないが、  
この場合を除外するためには別パターンのパラメタを用意する必要があるので、ここではそのままとする。  

【Scikit-LearnによるGrid Search実装方法】  
sklearn.model_selection.GridSearchCVクラスを用いる。  
1. インスタンス生成
   * estimator:　sklearnのモデルインスタンス
   * param_grid:　探索するパラメータと値の辞書型を指定
   * scoring:　評価指標
   * cv:　sklearnのcvオブジェクト
2. .fit(X, y)で学習
   * estimatorのfitメソッドの引数を **fit_paramsとして渡すことが可能
   * → これによりLightGBMのCallbacks等を渡すことが出来る。
3. .best_params_や.best_score_で結果を確認

In [39]:
# データセットを読み込み
data_set = fetch_california_housing()

In [40]:
# データセットの中身
data_set

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [41]:
#  特徴量と目的変数読み込み。yはnumpyのままで問題ない。
X = pl.DataFrame(data_set['data'], schema=data_set['feature_names'])
y = data_set['target']

In [42]:
X.dtypes

[Float64, Float64, Float64, Float64, Float64, Float64, Float64, Float64]

In [43]:
# 学習データと検証データに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

今回のデータはカテゴリ変数が含まれていないのでエンコーディングは不要。  
またGBDTなので標準化も不要。

In [44]:
# モデル定義
model = lgb.LGBMRegressor(random_state=0)
# GridSerchに使うパラメタを辞書型で定義
param_grid = {'num_leaves': [10, 20, 30, 40, 50, 60],
              'max_depth': [5, 10, 15],
              'reg_alpha': [0, 0.01, 0.03]}

cv = KFold(n_splits=3, random_state=0, shuffle=True)
# scoringに何も指定しない(default)と、LightGBM回帰の場合はRの2乗値になる。
gs = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv)


In [45]:
# early_stopping。今回検証データは固定とする。
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
# fit時の引数を定義
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}
# fit_paramsはアンパッキングオペレータ付きの辞書型で渡す。
gs.fit(X_train, y_train, **fit_params)


Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.264841
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.256456
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.255538
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240402
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.240164
Did not meet ea

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
             estimator=LGBMRegressor(random_state=0),
             param_grid={'max_depth': [5, 10, 15],
                         'num_leaves': [10, 20, 30, 40, 50, 60],
                         'reg_alpha': [0, 0.01, 0.03]})

過程を見ると、early stoppingし切れていないが、  
時間かかるので今回はデフォルトの100回で止める。  
hold outで作ったX_val, y_valを検証データとして固定しているので、early stoppingの過程でこの検証データに対して過学習気味になっている可能性があることに注意。  
上記ではeval_setを固定にしているが、これの是非については「機械学習に関するTips.ipynb」の「交差検証の際、GBDTのearly stopping用eval_setはCVの中で指定すべき？それとも外？」  
に自分の考えを書いておいた。実務を考えると、固定にするのはよくないのではと思う。

In [46]:
# GridSearchCVの結果を表示
# polarsだとparamsがstructになってしまい見づらいので、pandaで参照した方がわかりやすい
cv_results_df = pd.DataFrame(gs.cv_results_) 
cv_results_df.head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_num_leaves,param_reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.209834,0.069569,0.012622,0.006432,5,10,0,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.797096,0.807785,0.805461,0.803448,0.004590,54
1,0.167052,0.026787,0.006831,0.000743,5,10,0.01,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.798894,0.806981,0.806050,0.803975,0.003613,53
2,0.234722,0.132397,0.007434,0.001069,5,10,0.03,"{'max_depth': 5, 'num_leaves': 10, 'reg_alpha'...",0.799277,0.806364,0.806658,0.804100,0.003413,52


主に見るべき結果はparams（GridSearch後のパラメタ）、rank_test_score（CV結果の順位。１位のものが最も良いパラメタの組み合わせ）

In [47]:
# rank_test_scoreが5位以下の行のパラメタを表示
# iterrowsの結果はindex, Seriesのタプルになっている。
for index, row in cv_results_df[cv_results_df['rank_test_score'] <= 5].iterrows():
    print(f'{row["rank_test_score"]}: {row["params"]}')
    print(f'{row["mean_test_score"]}')

4: {'max_depth': 10, 'num_leaves': 50, 'reg_alpha': 0.01}
0.8329725903236045
3: {'max_depth': 15, 'num_leaves': 50, 'reg_alpha': 0}
0.8331058990226472
2: {'max_depth': 15, 'num_leaves': 50, 'reg_alpha': 0.03}
0.8332059773155042
5: {'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0}
0.8327661059213919
1: {'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0.01}
0.8338051795879502


In [91]:
# 最適なパラメタは下記でも取得可能
gs.best_params_

{'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0.01}

In [48]:
# 最適なパラメタに設定して学習データ全てで学習し、精度を見てみる。
# 学習率やn_estimatorsはチューニング対象外なので、極端に低め or 高めに設定する。
model_opt = lgb.LGBMRegressor(**gs.best_params_, random_state=0, learning_rate=0.01, n_estimators=1000)
# early_stopping
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
# fit時の引数を定義
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}

# lgbmの学習。fit_paramsはアンパッキングオペレータ付きの辞書型で渡す。
model_opt.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.195693


LGBMRegressor(learning_rate=0.01, max_depth=15, n_estimators=1000,
              num_leaves=60, random_state=0, reg_alpha=0.01)

In [49]:
model_opt.score(X_val, y_val)

0.8499232425233268

CV時よりも精度が少し高い結果となった。  
これは学習データ数増加、学習率減、n_estimatorsが増したためと思われる。  
ただし、上記はearly stoppingに使用した検証データに対するスコアであるため、検証データに過学習気味の結果であることに注意。  
実務ではテストデータに対するスコアが重要である。  
また、early stoppingし切れていないのでより精度を求める場合はしっかりearly stoppingが確認できるまでn_estimatorsを増やす必要がある。

## GridSearchCVにPipelineを組み込む
GridSearchCVのestimatorにはmodelを含むPipelineオブジェクトを入れることが出来るので、  
ダミー変数化、標準化と合わせてチューニング可能。  
ここではLightGBMを用いる。ML_notebook-17で苦労したが、同様の方法でcross_val_scoreでLightGBM使えるかも。

In [83]:
dtypes = {
    "species": str,
    'island': str,
    'culmen_length_mm': pl.Float32, # くちばしの長さ[mm]
    'culmen_depth_mm': pl.Float32, # くちばしの高さ[mm]
    'flipper_length_mm': pl.Float32, # 翼の長さ[mm]
    'body_mass_g': pl.Float32, # 体重[g]
    'sex': str
}

# ペンギンのデータセット読み込み。欠損値がNAとして含まれているので、null_values="NA"を指定しないと読み込みエラーになる。
df = pl.read_csv('../Python/sample_data/ML_sample/penguins_size.csv',dtypes=dtypes, null_values='NA')

### - 前処理

In [84]:
# sexカラムの.は欠損値扱いとする。
df = df.with_columns(
    (pl.when(pl.col('sex') == '.').then(None).otherwise(pl.col('sex'))).alias('sex')
)

# 欠損値が多すぎる行（値が入っている列が3つより少ない行）を削除する。
# この操作はPolarsだと面倒なので、一回Pandasに変換してやる。
df = pl.from_pandas(df.to_pandas().dropna(thresh=3))

In [85]:
df.null_count()

species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,9


In [86]:
# 特徴量と目的変数定義
target = 'species'
X = df.drop(target)
y = df[target]

In [87]:
# 量的変数と質的変数のカラム名をそれぞれ取得。目的変数は抜く。
num_feat = df.select(pl.col(pl.Float32)).columns
cat_feat = df.select(pl.col(pl.Utf8)).drop(target).columns

### - Pipeline用意

In [88]:
# 欠損値対応。今回は最頻値で埋める。GBDTなのでなくてもよいかも。
# cat_feat以外のカラムを落とさないためにremainder='passthrough'を指定。
imputer = SimpleImputer(strategy='most_frequent') 

# 欠損値対応とダミー変数化を同時に行うためにカテゴリ変数に対する処理をまとめる。CTを使うので、今回は自作GetDummiesは使わない。
pipeline_for_cat = Pipeline([('impute_cat', SimpleImputer(strategy='most_frequent')),
                             ('OHE', OneHotEncoder(handle_unknown='ignore', sparse_output=False))])

ct = ColumnTransformer([('impute_cat+OHE', pipeline_for_cat, cat_feat)], remainder='passthrough')
ct.set_output(transform='pandas')

# モデル定義。今回はearly_stoppingしない。
# する場合は前回のようにtrain_test_splitで学習データと検証データを分ける必要がある。
lgbmc = lgb.LGBMClassifier(random_state=0)

# 最終的なPipeline定義
pipeline = Pipeline(steps=[('impute_cat+OHE', ct), ('model', lgbmc)])


In [89]:
# ColumnTransformerの確認
ct.fit_transform(X.to_pandas()).head()

,impute_cat+OHE__island_Biscoe,impute_cat+OHE__island_Dream,impute_cat+OHE__island_Torgersen,impute_cat+OHE__sex_FEMALE,impute_cat+OHE__sex_MALE,impute_cat+OHE__sex_None,remainder__culmen_length_mm,remainder__culmen_depth_mm,remainder__flipper_length_mm,remainder__body_mass_g
0,0.0,0.0,1.0,0.0,1.0,0.0,39.099998,18.700001,181.0,3750.0
1,0.0,0.0,1.0,1.0,0.0,0.0,39.500000,17.400000,186.0,3800.0
2,0.0,0.0,1.0,1.0,0.0,0.0,40.299999,18.000000,195.0,3250.0
3,0.0,0.0,1.0,1.0,0.0,0.0,36.700001,19.299999,193.0,3450.0
4,0.0,0.0,1.0,0.0,1.0,0.0,39.299999,20.600000,190.0,3650.0


In [90]:
pipeline

Pipeline(steps=[('impute_cat+OHE',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_cat+OHE',
                                                  Pipeline(steps=[('impute_cat',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OHE',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['island', 'sex'])])),
                ('model', LGBMClassifier(random_state=0))])

### - GridSearchCV

In [91]:
# cvの定義
cv = KFold(n_splits=3, random_state=0, shuffle=True)

# GridSearchの定義。pipelineをestimatorとして入れる。
# pipelineのどのステップのパラメタなのかを指定するために、param_gridの各値にはステップ名（上記では'model__'）を接頭辞に付ける。
param_grid = {'model__num_leaves': [10, 20, 30, 40, 50, 60],
              'model__max_depth': [5, 10, 15],
              'model__reg_alpha': [0, 0.01, 0.03]}


gs = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=cv)
gs.fit(X.to_pandas(), y.to_pandas())

GridSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
             estimator=Pipeline(steps=[('impute_cat+OHE',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_cat+OHE',
                                                                         Pipeline(steps=[('impute_cat',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('OHE',
                                                                                          OneHotEncoder(handle_unknown='ignore',
                                                                                                        sparse_output=False))]),
                                                                         ['island',
                                                                          'sex'])])),
                                       ('model',
                                        LGBMClassifier(random_state=0))]),
             param_grid={'model__max_depth': [5, 10, 15],
                         'model__num_leaves': [10, 20, 30, 40, 50, 60],
                         'model__reg_alpha': [0, 0.01, 0.03]})

In [95]:
# 結果の確認
cv_results_df = pl.DataFrame(gs.cv_results_)
cv_results_df

mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__max_depth,param_model__num_leaves,param_model__reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
f64,f64,f64,f64,object,object,object,struct[3],f64,f64,f64,f64,f64,i32
0.166645,0.07826,0.013913,0.002757,5,10,0,"{5,10,0}",0.991228,0.973684,0.982456,0.982456,0.007162,31
0.072721,0.007702,0.013968,0.002347,5,10,0.01,"{5,10,0}",0.991228,0.982456,0.982456,0.98538,0.004135,1
0.269807,0.275962,0.012215,0.000925,5,10,0.03,"{5,10,0}",0.991228,0.982456,0.973684,0.982456,0.007162,31
0.194933,0.122723,0.012214,0.000553,5,20,0,"{5,20,0}",0.991228,0.973684,0.982456,0.982456,0.007162,31
0.065055,0.010715,0.016252,0.006013,5,20,0.01,"{5,20,0}",0.991228,0.982456,0.982456,0.98538,0.004135,1
0.079504,0.022891,0.016095,0.004697,5,20,0.03,"{5,20,0}",0.991228,0.982456,0.973684,0.982456,0.007162,31
0.099038,0.014517,0.012735,0.000517,5,30,0,"{5,30,0}",0.991228,0.973684,0.982456,0.982456,0.007162,31
0.064031,0.00251,0.017451,0.006101,5,30,0.01,"{5,30,0}",0.991228,0.982456,0.982456,0.98538,0.004135,1
0.068352,0.014107,0.015292,0.003979,5,30,0.03,"{5,30,0}",0.991228,0.982456,0.973684,0.982456,0.007162,31


rank_test_score（今回はデフォルトなので、平均accuracy）が1か31しかない。  
すなわちパラメタを変えても２通りにしかスコア(ここではaccuracy)が変わらないということ。  
このデータは簡単な分類タスクなので、高精度にしかならないのかも。  
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html

In [94]:
# ベストな組み合わせを確認
gs.best_params_

{'model__max_depth': 5, 'model__num_leaves': 10, 'model__reg_alpha': 0.01}

## Random Search
sklearn.model_selection.RandomizedSearchCVクラスを使う。
1. インスタンス生成
   * estimator: sklearnのモデルインスタンス
   * param_distributions: 探索するハイパーパラメタと値を出力するobject(または値のリスト)のdict
     * 例: {”パラメタ名”: object, …}
       * objectは.rvs()メソッドを実装している必要がある。
       * scipy.stats.uniformやscipy.stats.randintを使用することが多い。
       * このように確率分布を指定してランダムなパラメタを選ぶ
   * scoring: 評価指標
   * cv: sklearnのcvオブジェクト
   * n_iter: イテレーションの回数
2. .fit(X, y)で学習
3. .best_params_や.best_score_で結果を確認

In [51]:
# データセットを読み込み
data_set = fetch_california_housing()
#  特徴量と目的変数読み込み。yはnumpyのままで問題ない。
X = pl.DataFrame(data_set['data'], schema=data_set['feature_names'])
y = data_set['target']
# 学習データと検証データに分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

# モデル定義
model = lgb.LGBMRegressor(random_state=0)
# チューニング対象パラメタと確率分布の定義。どちらも一様分布。
param_dist = {'num_leaves': sp_randint(10, 60),
              'max_depth': sp_randint(5, 15),
              'reg_alpha': uniform(0, 0.03)}

callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}

# cv
cv = KFold(n_splits=3, random_state=0, shuffle=True)

In [53]:
# random serch。今回はランダムにパラメタを選んで評価を30回繰り返す。
rscv = RandomizedSearchCV(estimator=model, param_distributions=param_dist, cv=cv, n_iter=30)
rscv.fit(X_train, y_train, **fit_params)

Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 0.198816

RandomizedSearchCV(cv=KFold(n_splits=3, random_state=0, shuffle=True),
                   estimator=LGBMRegressor(random_state=0), n_iter=30,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3f1d3528e0>,
                                        'num_leaves': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3fc85ab340>,
                                        'reg_alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7f3f1d9538b0>})

上記のGrid Searchの例ではイテレーション回数が54回だったので、Random Searchの方が半分程度の時間で早く終わっている。  

In [62]:
# 結果を表示
cv_results_df = pd.DataFrame(rscv.cv_results_)
cv_results_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_num_leaves,param_reg_alpha,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.395376,0.179364,0.013854,0.003447,9,29,0.022578,"{'max_depth': 9, 'num_leaves': 29, 'reg_alpha'...",0.822364,0.832022,0.824378,0.826255,0.004160,17
1,0.343545,0.022955,0.014056,0.006256,12,45,0.026248,"{'max_depth': 12, 'num_leaves': 45, 'reg_alpha...",0.828312,0.833028,0.829364,0.830234,0.002021,7
2,0.301701,0.044576,0.009982,0.002067,8,31,0.014996,"{'max_depth': 8, 'num_leaves': 31, 'reg_alpha'...",0.823163,0.830717,0.826410,0.826764,0.003094,15
3,0.203696,0.012147,0.019410,0.005573,10,23,0.013779,"{'max_depth': 10, 'num_leaves': 23, 'reg_alpha...",0.821207,0.826500,0.823473,0.823727,0.002168,23
4,0.203293,0.056662,0.017732,0.013126,8,15,0.007256,"{'max_depth': 8, 'num_leaves': 15, 'reg_alpha'...",0.812475,0.820931,0.818399,0.817268,0.003543,26


In [55]:
# rank_test_scoreが5位以下の行のパラメタを表示
# iterrowsの結果はindex, Seriesのタプルになっている。
for index, row in cv_results_df[cv_results_df['rank_test_score'] <= 5].iterrows():
    print(f'{row["rank_test_score"]}: {row["params"]}')
    print(f'{row["mean_test_score"]}')

4: {'max_depth': 12, 'num_leaves': 42, 'reg_alpha': 0.006238809690722126}
0.8313630586227037
5: {'max_depth': 12, 'num_leaves': 37, 'reg_alpha': 0.012845732567399555}
0.8308357099560411
2: {'max_depth': 13, 'num_leaves': 52, 'reg_alpha': 0.020586372067895015}
0.8337141954542956
3: {'max_depth': 14, 'num_leaves': 54, 'reg_alpha': 0.010562352034623464}
0.8318774471319297
1: {'max_depth': 14, 'num_leaves': 56, 'reg_alpha': 0.019031270203414056}
0.8340552355056512


※参考　Grid Searchの時のパラメタ

1: {'max_depth': 15, 'num_leaves': 60, 'reg_alpha': 0.01}  
0.8338051795879502 

random search CVとGrid Search CVでそんなに差はなかった。  
わずかにrandom searchの方が良い。  
ただ、Random searchの方がGrid Searchでは定義できない組み合わせで試行することによって、  
精度が出るパラメタを見つけられることがある。

In [58]:
# 最も良いパラメタ
rscv.best_params_

{'max_depth': 14, 'num_leaves': 56, 'reg_alpha': 0.019031270203414056}

In [60]:
# 最適なパラメタに設定して学習データ全てで学習し、精度を見てみる。
# 学習率やn_estimatorsはチューニング対象外なので、極端に低め or 高めに設定する。
model_opt = lgb.LGBMRegressor(**rscv.best_params_, random_state=0, learning_rate=0.01, n_estimators=1000)
# early_stopping
callbacks = [lgb.early_stopping(stopping_rounds=10)]
eval_set = [(X_val, y_val)]
# fit時の引数を定義
fit_params = {'callbacks': callbacks, 'eval_set': eval_set}

# lgbmの学習。fit_paramsはアンパッキングオペレータ付きの辞書型で渡す。
model_opt.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l2: 0.197797


LGBMRegressor(learning_rate=0.01, max_depth=14, n_estimators=1000,
              num_leaves=56, random_state=0, reg_alpha=0.019031270203414056)

In [61]:
model_opt.score(X_val, y_val)

0.8483099817275959

## ベイズ最適化
hyperoptライブラリを使う。（pip install hyperoptする必要あり）
* hyperopt.fmin.fmin()で最適化を実行
  * objective: 最小化したい目的関数(pythonの関数)を返すfunctionオブジェクトを渡す。
    * モデルインスタンス生成
    * 学習および評価
    * 精度（損失）を返す
  * space: パラメタの範囲をdictitonary形式で渡す
    * hyperopt.hp.quniform()や hyperopt.hp.uniform()を使用した確率分布でパラメタを渡す。
  * algo: hyperopt.tpe.suggestを渡す。
  * max_evals: イテレーション回数

In [77]:
# データセットを読み込み
data_set = fetch_california_housing()
#  特徴量と目的変数読み込み。yはnumpyのままで問題ない。
X = pl.DataFrame(data_set['data'], schema=data_set['feature_names'])
y = data_set['target']


# ベイズ最適化に入れる関数f(x)を定義する。
# f(x)の実態はハイパーパラメータを入力として、モデルを学習し、スコアを返す関数である。
# (数式としてブラックボックスだが損失を計算する仕組みはもちろん定義する必要がある）
def objective(params):
    # hp.quniformの結果はfloatなのでLGBMに入れる際はintにキャストする必要がある。
    params = {'num_leaves': int(params['num_leaves']),
              'max_depth': int(params['max_depth']),
              'reg_alpha': params['reg_alpha']}

    # モデル定義。early stoppingは今回しない。
    model = lgb.LGBMRegressor(**params, random_state=0)
    cv = KFold(n_splits=3, random_state=0, shuffle=True)
    scores = cross_val_score(model, X, y, cv=cv)

    # ログ記録
    log['params'].append(params)
    log['score'].append(scores.mean())
    log['score_std'].append(scores.std())

    # LGBM回帰のデフォルトスコアはR squaredであり、これは大きい方が良いので、マイナスをつける。
    return -scores.mean()
    

# チューニング対象パラメタと確率分布の定義。quniformについては公式ドキュメント参照。randintに近いもの。
space = {'num_leaves': hp.quniform('num_leaves', 10, 60, 2),
              'max_depth': hp.quniform('max_depth', 5, 15, 2),
              'reg_alpha': hp.uniform('reg_alpha', 0, 0.03)}


# hyperoptではログが出力されないので、ログの機構を実装する
log = {'params': [], 'score': [], 'score_std': []}



In [78]:
# ベイズ最適化実行
best_param = fmin(objective, space=space, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [01:24<00:00,  1.19trial/s, best loss: -0.8415549944859705]


In [79]:
# 最終的なベストパラメタを確認
best_param

{'max_depth': 12.0, 'num_leaves': 60.0, 'reg_alpha': 0.010867306025058641}

In [97]:
# ログを確認。
log_df = pl.DataFrame(log)
log_df.sort('params', descending=True).head()

params,score,score_std
struct[3],f64,f64
"{60,14,0.00718}",0.840486,0.003042
"{60,14,0.00697}",0.840522,0.003015
"{60,14,0.00657}",0.839731,0.002718
"{60,12,0.011783}",0.84129,0.003133
"{60,12,0.010867}",0.841555,0.003101


Polarsでは辞書のキーとバリューが表示されず、  
見づらいので元のログを使って見やすくする。

In [98]:
log_df = log_df.with_columns(
    pl.DataFrame(log['params'])
).drop('params') # 不要なので削除

In [100]:
log_df.sort('score', descending=True).head()

score,score_std,num_leaves,max_depth,reg_alpha
f64,f64,i64,i64,f64
0.841555,0.003101,60,12,0.010867
0.84129,0.003133,60,12,0.011783
0.840981,0.00284,56,14,0.008068
0.840962,0.002846,56,14,0.007988
0.840882,0.002722,56,14,0.003473
